In [1]:

from sklearn.linear_model import LinearRegression
import joblib
import pickle
import datetime,time
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
#import seaborn as sns
#sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.cluster import KMeans
#import matplotlib.pyplot as plt
#plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV
from lightgbm import LGBMRegressor,LGBMClassifier
#from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
le=LabelEncoder()
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
import warnings
warnings.filterwarnings("ignore")
#from tqdm.notebook import tqdm_notebook

In [2]:
import os
df = pd.read_csv('Data/sample.csv')
col=df.drop(["leak_point","leak_distance_(metre)"],axis=1).columns
df

,pressure_in,flow_in,pressure_out,flow_out,leak_point,leak_distance_(metre)
0,39.849290,0.230226,40.750116,0.118118,1.0,15.0
1,39.976823,0.233183,40.309626,0.121653,1.0,15.0
2,40.086820,0.235243,40.920886,0.123221,1.0,15.0
3,39.962475,0.230033,40.547427,0.117295,1.0,15.0
4,40.203194,0.228338,40.624034,0.118521,1.0,15.0
...,...,...,...,...,...,...
8995,39.264530,0.257558,38.514566,0.167770,3.0,50.0
8996,40.070877,0.259461,38.334725,0.167491,3.0,50.0
8997,39.730525,0.250250,38.697590,0.160487,3.0,50.0
8998,39.940462,0.259921,38.665760,0.170681,3.0,50.0


In [3]:
px.scatter(data_frame=df,y='leak_distance_(metre)',x='pressure_in',hover_name='leak_point')

In [4]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing_num'] = df.isna().sum().values*100
    sum['missing%'] = (df.isna().sum().values*100)/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    desc = pd.DataFrame(df.describe().T)
    sum['min'] = desc['min']
    sum['max'] = desc['max']
    sum['mean'] = desc['mean']
    sum['75%'] = desc['75%']
    return sum

summary(df).style.background_gradient(cmap='Blues')

,dtypes,missing_num,missing%,uniques,count,min,max,mean,75%
pressure_in,float64,0,0.000000,2364,9000,37.281264,40.990146,39.371006,40.045372
flow_in,float64,0,0.000000,8902,9000,0.199725,0.813442,0.397171,0.686630
pressure_out,float64,0,0.000000,2203,9000,28.766561,41.294345,36.393231,40.323990
flow_out,float64,0,0.000000,8940,9000,0.085826,0.543492,0.237825,0.411126
leak_point,float64,0,0.000000,3,9000,1.000000,3.000000,2.000000,3.000000
leak_distance_(metre),float64,0,0.000000,3,9000,15.000000,50.000000,31.666667,50.000000


In [5]:

df.fillna(df.mean(),inplace=True)

In [6]:
#checking for correlations,which col to be droprd or maintain

top20_corrs = abs(df.corr()['flow_out']).sort_values(ascending = False).head(30)
top20_corrs

flow_out                 1.000000
flow_in                  0.994212
pressure_out             0.985018
pressure_in              0.906564
leak_distance_(metre)    0.050602
leak_point               0.000052
Name: flow_out, dtype: float64

In [ ]:
# Trying to remove redundant features
def redundant_feature(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 99:
            redundant.append(i)
    redundant = list(redundant)
    return redundant
redundant_features = redundant_feature(df)
redundant_features

In [ ]:
df.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
df['flow_out'].value_counts()

# modelling, variation of validation and so on----

In [ ]:

from sklearn.model_selection  import train_test_split
X=df[col]
y = df["leak_point"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [ ]:
# ctb = CatBoostRegressor(**cn).fit(X,y)
# xgr = XGBRegressor(**xn).fit(X,y)
lgr = LGBMRegressor().fit(X,y)
# from sklearn.multioutput import RegressorChain,ClassifierChain,ClassifierMixin,RegressorMixin,MultiOutputRegressor 
# model_ = MultiOutputRegressor(estimator=RandomForestRegressor().fit(X,y))
# model_.fit(X,y)

In [ ]:
y_predicted = lgr.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, (y_predicted)))
print("Root Mean Square Error:\n")
print(RMSE)


In [ ]:

#pickle.dump(lgr2, open('./models/lightgbm_1.pkl','wb'))
joblib.dump(lgr, './models/lightgbm_1.pkl')

# second model for leak_distance

In [ ]:
y2 = df["leak_distance_(metre)"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.33, random_state=123)
lgr2 = LGBMRegressor().fit(X,y2)

In [ ]:
y_predicted = lgr2.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, (y_predicted)))
print("Root Mean Square Error:\n")
print(RMSE)

In [ ]:
pickle.dump(lgr2, open('./models/lightgbm_2.pkl','wb'))

In [ ]:

#joblib.dump(lgr2, './models/lightgbm_2.pkl')

In [ ]:

import optuna
from catboost import CatBoostRegressor
X_train, X_valid, y_train, y_valid= train_test_split(X, y, test_size=0.2, random_state=123)
# Define the objective function for Optuna
def catboost_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 10)
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(catboost_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"CatBoost Best Hyperparameters: {best_params}")
print(f"CatBoost Best RMSE: {best_rmse}")#513


In [ ]:
import optuna
import xgboost as xgb

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def xgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(xgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"XGBoost Best Hyperparameters: {best_params}")
print(f"XGBoost Best RMSE: {best_rmse}")#489


In [ ]:

import lightgbm as lgb
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def lgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100)
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train,y_train,eval_metric='rmse')
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(lgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"LightGBM Best Hyperparameters: {best_params}")
print(f"LightGBM Best RMSE: {best_rmse}")#491

In [ ]:
kf = KFold(n_splits=5,shuffle=False)
model = CatBoostRegressor(**c)
RMSE = []
R2=[]
c_train_pred = np.zeros(len(X))
c_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_test,y_test)],verbose=False)
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    c_train_pred+=model.predict(X)
    c_test_pred+=model.predict(test[col])
    c_predictions= np.absolute(c_test_pred/kf.n_splits)
    sub['power-generated']=c_predictions
    sub.to_csv('ctb_cv5.csv',index=False)


print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #
f"Root Mean Squared Error: {np.mean(RMSE)} ")# 2896

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor(**x)
kf = KFold(n_splits=5, shuffle=False)
RMSE = []
R2 = []
x_train_pred = np.zeros(len(X))
x_test_pred = np.zeros(len(test))

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train, early_stopping_rounds=500, eval_set=[(X_test, y_test)], verbose=False)
    predictions = model.predict(X_test)
    RMSE.append(np.sqrt(mean_squared_error(predictions, y_test)))
    R2.append(r2_score(predictions, y_test))
    x_train_pred[train_index] = model.predict(X_train)
    x_test_pred += model.predict(test[X_train.columns])
x_predictions = np.absolute(x_test_pred / kf.get_n_splits())
sub['power-generated'] = x_predictions
sub.to_csv('xgb_cv5.csv', index=False)
print(f"r2 or accuracy: {np.mean(R2)}")
print(f"Root Mean Squared Error: {np.mean(RMSE)}")


In [ ]:
kf = KFold(n_splits=5,shuffle=False)
model = LGBMRegressor(**l)
RMSE = []
R2=[]
l_train_pred = np.zeros(len(X))
l_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,eval_metric='rmse')
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    l_train_pred+=model.predict(X)
    l_test_pred+=model.predict(test[col])
    l_predictions= np.absolute(l_test_pred/kf.n_splits)
    sub['power-generated']=l_predictions
    sub.to_csv('lgr_cv5.csv',index=False)
print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #591322.5448032819
f"Root Mean Squared Error: {np.mean(RMSE)} ")#